In [ ]:
import json
import requests
import ipywidgets as widgets

from IPython.display import display

In [ ]:
# Add heading
heading = widgets.HTML("<h2>Credentials</h2>")
display(heading)

domain_input = widgets.Text(description='Username:', placeholder="i001ua.tryezmeral.com")
username_input = widgets.Text(description='Username:')
password_input = widgets.Password(description='Password:')
submit_button = widgets.Button(description='Submit')
success_message = widgets.Output()

domain = None
username = None
password = None

def submit_button_clicked(b):
    global domain, username, password
    domain = domain_input.value
    username = username_input.value
    password = password_input.value
    with success_message:
        success_message.clear_output()
        print("Credentials submitted successfully!")
    submit_button.disabled = True

submit_button.on_click(submit_button_clicked)

# Set margin on the submit button
submit_button.layout.margin = '20px 0 20px 0'

# Display inputs and button
display(domain_input, username_input, password_input, submit_button, success_message)

In [ ]:
token_url = f"https://keycloak.{domain}/realms/UA/protocol/openid-connect/token"

data = {
    "username" : username,
    "password" : password,
    "grant_type" : "password",
    "client_id" : "ua-grant",
}

token_responce = requests.post(token_url, data=data, allow_redirects=True, verify=False)

token = token_responce.json()["access_token"]

In [ ]:
DOMAIN_NAME = "svc.cluster.local"  # change this to your domain for external access
NAMESPACE = username
DEPLOYMENT_NAME = "vllm-model"
MODEL_NAME = "vllm_model"
SVC = f'{DEPLOYMENT_NAME}-predictor-default.{NAMESPACE}.{DOMAIN_NAME}'
URL = f"https://{SVC}/v2/models/{MODEL_NAME}/infer"

print(URL)

In [ ]:
inference_request = {
    "inputs" : [{
        "name" : "text_input",
        # "datatype": "FP32",

        "shape": [1],
        # Example of non-fraudulent Transaction Dtls
        # "data": [list(item) for item in X.values][14],
        # Example of a fraudulent request
        "data": ["What is Triton Inference Server?"],
    }]
}

print("data:", inference_request)

In [ ]:
headers = {"Authorization": f"Bearer {token}"}
response = requests.post(URL, json=inference_request, headers=headers, verify=False)

In [ ]:
response.text